In [30]:
import os
import sys
while not os.getcwd().endswith('ml'):
    os.chdir('..')
sys.path.insert(0, os.getcwd())

In [31]:
import math
import copy
import cv2
import numpy as np
import random
from shutil import copyfile
from pathlib import Path
from matplotlib import pyplot as plt
from kaggle_problems.rosneft_proppant.workspace import helpers
from kaggle_problems.rosneft_proppant.workspace.helpers import *
helpers.circleContour = GrayCircleContour()

In [34]:
def process_bw_img(original_img, img_number):
    result_circles = []
    
    debug_dir = "kaggle_problems/rosneft_proppant/data/debug/{}".format(img_number)
    Path(debug_dir).mkdir(exist_ok=True, parents=True)
    
    img_with_circle = copy.deepcopy(original_img)
    img = copy.deepcopy(original_img)

    gaussian_3 = cv2.GaussianBlur(img, (0, 0), 2.0)
    img = cv2.addWeighted(img, 1.5, gaussian_3, -0.5, 0, img)
    cv2.imwrite("{}/img.jpg".format(debug_dir), img)

    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    th, threshed = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

    it = 0
    for r in range(20, 7, -1):
        print('-' * 30)
        # Выделение кругов
        circles = cv2.HoughCircles(threshed,cv2.HOUGH_GRADIENT,0.5,minDist=r,
                                param1=200,param2=1,minRadius=r,maxRadius=r)
        circles = circles[0]
        print("Circle count: {}. r: {}".format(len(circles), r))
        filtered_circle = [(int(circle[0]), int(circle[1]), int(circle[2])) for circle in circles]

        filtered_circle = [circle for circle in filtered_circle \
        if is_convex_in_circle(threshed, circle[0], circle[1], circle[2])]

        print("After first filter: {}. r: {}".format(len(filtered_circle), r))

        filtered_circle = [circle for circle in filtered_circle \
            if is_border_white_circle(threshed, circle[0], circle[1], circle[2])]

        print("After second filter: {}. r: {}".format(len(filtered_circle), r))

        for circle in filtered_circle:
            cv2.circle(threshed,(circle[0],circle[1]),circle[2], 0, -1) # fill with 0 because of THRESH_BINARY_INV
            cv2.circle(original_img,(circle[0],circle[1]),circle[2],(255, 255, 255),-1)
            cv2.circle(img_with_circle,(circle[0],circle[1]),circle[2], get_random_color(),1)

        cv2.imwrite("{}/original_img_{}.jpg".format(debug_dir, it), original_img)
        cv2.imwrite("{}/img_with_circle_{}.jpg".format(debug_dir, it), img_with_circle)
        cv2.imwrite("{}/threshed_{}.jpg".format(debug_dir, it), threshed)
        it += 1
        result_circles.extend(filtered_circle)
        
    return result_circles

In [37]:
img_name = "6"
original_img = cv2.imread("kaggle_problems/rosneft_proppant/data/processing/main_area/{}.jpg".format(img_name)) 

#process_bw_img(original_img, img_name)


ModuleNotFoundError: No module named 'helpers'

In [39]:
all_img = [img for img in os.listdir(ORIGINAL_IMG_DIR) if img.endswith('.jpg') ]

for img_name in all_img:
    img_number, _ = img_name.split('.')
    original_img = cv2.imread("{}/{}".format(ORIGINAL_IMG_DIR, img_name)) 
    Path("{}/processing/bw_images".format(DATA_DIR)).mkdir(exist_ok=True, parents=True)
    Path("{}/processing/color_images".format(DATA_DIR)).mkdir(exist_ok=True, parents=True)
    
    if is_grey_img(original_img):
        copyfile("{}/{}".format(ORIGINAL_IMG_DIR, img_name), "{}/processing/bw_images/{}".format(DATA_DIR, img_name))
    else:
        copyfile("{}/{}".format(ORIGINAL_IMG_DIR, img_name), "{}/processing/color_images/{}".format(DATA_DIR, img_name))


KeyboardInterrupt: 